<a href="https://colab.research.google.com/github/martinhavlicek/OHBM-hackathons-on-using-DNNs/blob/master/Defacing_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras for Neuroimaging

Author: J. Andrew Doyle, Montreal Neurological Institute

First presented at:

[Montreal Artificial Intelligence & Neuroscience](http://www.crm.umontreal.ca/2018/MAIN2018/index_e.php): Hands-on Deep Learning with Keras workshop

December 10, 2018

Centre de Recherches Mathématiques, Université de Montréal

Contact: [andrew.doyle@mcgill.ca](mailto:andrew.doyle@mcgill.ca)    Twitter: [@crocodoyle](https://twitter.com/crocodoyle)

![Sponsor logos](https://amnesia.cbrain.mcgill.ca/deeplearning/img/sponsor_logos.png)

# 1. Goal

Properly **anonymizing data** is essential to be able to share data and collaborate. This notebook will demonstrate how to use [Keras](https://keras.io), a high-level deep learning library, to automatically **detect whether a T1w MRI scan has been defaced** or not.

This notebook was inspired by the [Neuroinformatics & Google Summer of Code Defacing Detector project](https://summerofcode.withgoogle.com/archive/2018/projects/5287412167081984/). For that project, a more complex 2.5 D convolutional neural network was trained by Wazeer Zulfikar (co-mentored by Chris Gorgolewski). The trained neural network is currently being integrated into the [BIDS-Validator](https://github.com/bids-standard/bids-validator) that performs automatic checks that neuroimaging datasets are ready to share!

The dataset prepared for this tutorial is a 2D, slimmed down version of [IXI](https://brain-development.org/ixi-dataset/), where participants are all healthy. It is the only publicly-available dataset I know of that includes faces of participants. This smaller version consists of sagittal slices of (1) the original MRI scans, and (2) "defaced" images, where [pydeface](https://github.com/poldracklab/pydeface) has been run to remove identifiable features from subjects' brain scans.

![example of image original and defaced MRI slices](https://amnesia.cbrain.mcgill.ca/deeplearning/img/defacing_example.png)



Most machine learning techniques are probabilistic in nature. For this notebook, are are trying to estimate the probability that an image has been defaced. Let *D* represent a random variable where *D=0* denotes that defacing has *not* been done, and *D=1* means that defacing has been properly applied. *MRI* is a continuous-valued random field with 2D spatial structure that contains information about *D*. We wish to estimate the probability of *D* from our measurement *MRI*:

> *P ( D | MRI )*

More specifically, we will train **convolutional neural networks ** to compare the probabilities:


>*P ( D=1 | MRI )* and *P ( D=0 | MRI )*

Later on, *D* is refered to as `labels` and *MRI* as `images`

![alt text](https://amnesia.cbrain.mcgill.ca/deeplearning/img/morpheus.png)

First, install Keras using `pip`, the most popular package manager for Python. We're also going to install [Scikit-Learn](https://scikit-learn.org/stable/) and use a few of its utilities, and download and unzip the dataset for this tutorial. Select the cell with your mouse or arrow keys, and hit Shift + Enter to run the code.

In [ ]:
!pip install keras sklearn
!wget https://amnesia.cbrain.mcgill.ca/deeplearning/ixi_slices.tar.gz --no-check-certificate
!tar -xvf ixi_slices.tar.gz

# data is split into two folders:
# /slices/original/
# /slices/defaced/

# 2. Loading Data
The data in this example has two parts, the **images** (x) and a **label** (y) for whether it has been defaced or not. We can create the labels array based on which folder the scan is found in.

![alt text](https://amnesia.cbrain.mcgill.ca/deeplearning/img/files.png)

In [ ]:
from glob import glob        # tool to match file patterns
import numpy as np

n_classes = 2

original_images = glob('slices/original/*.jpg')
defaced_images = glob('slices/defaced/*.jpg')

# build labels array, starting as a list
# you could probably do this more efficiently with list comprehensions
# but I find this more readable
labels = []

for i in original_images:
  labels.append(0)           # D=0 for original, undefaced

for i in defaced_images:
  labels.append(1)           # D=1 for defaced
  
labels = np.uint8(labels)
image_filenames = original_images + defaced_images

n_images = len(labels)
n_channels = 3               # colour channels

print('Created list with labels. There are', n_images, 'of them.')

Now we have a list of filenames for *MRI* and a `numpy` array of integers for *D*, with matched indices.
Next, we transform the *D* into "one-hot encoding", where integer categorical labels are represented as a probability vector:

> *D=0* -> `[1, 0]`

> *D=1* -> `[0, 1]`

In [ ]:
from keras.utils import to_categorical
labels_one_hot = to_categorical(labels, num_classes=2)

print('First label (original, one-hot encodings):', labels[0], labels_one_hot[0])
print('Last label (original, one-hot encodings):', labels[-1], labels_one_hot[-1])

print('Labels shape in one-hot encoding:', labels_one_hot.shape)

Then we load the image data. It's currently stored as .jpg in `uint8` (0 - 255), and GPUs work with `float32`, so we convert to float and rescale between 0 -1. This linear rescaling is a fairly standard approach in deep learning, and we avoid whitening the data so that it has zero mean and unit standard deviation, like in most other machine learning methods.

![Normalizing data](https://amnesia.cbrain.mcgill.ca/deeplearning/img/normalize.png)

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import skimage   # image processing lib. OpenCV good too. ITK for the bold

image_data = np.zeros((n_images, 224, 224, n_channels), dtype=np.float32)

for i, img_filename in enumerate(image_filenames):
  img = plt.imread(img_filename)
  img = skimage.img_as_float(img)
  img = img / np.max(img)
  
  image_data[i, :, :, :] = img

  
print('First image (not-defaced):')
plt.imshow(image_data[0, :, :, :])
plt.axis('off')
plt.show()

print('Last image (defaced):')
plt.imshow(image_data[-1, :, :, :])
plt.axis('off')
plt.show()

## Train / Test Split 
At this point we should probably split up our data into a real training and test set. The `sklearn` package has some [very nice functions](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) for splitting data into training(/validation)/testing sets. We're going to use the most basic one, `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split

image_data_train, image_data_test, labels_train, labels_test = train_test_split(image_data, labels_one_hot, test_size=0.1, random_state=42)

# when in doubt, print all the shapes
# print all the shapes when not in doubt too
print('Training images shape:', image_data_train.shape)
print('Training labels shape:', labels_train.shape)

print('Test images shape:', image_data_test.shape)
print('Test labels shape:', labels_test.shape)

# check that the distribution of faced/defaced data is similar
label_integers_train = np.argmax(labels_train, axis=-1)
label_integers_test = np.argmax(labels_test, axis=-1)

# make a histogram
f, axes = plt.subplots(1, 2, sharey=True)
axes[0].hist(label_integers_train)
axes[1].hist(label_integers_test)

axes[0].set_xlabel('Train', fontsize=20)
axes[1].set_xlabel('Test', fontsize=20)

for ax in axes:
  ax.set_xticks([0, 1])
  ax.set_xticklabels(['Undefaced', 'Defaced'], fontsize=16)
  
plt.tight_layout()
plt.show()

# 3. Sequential API
The [Sequential API](https://keras.io/getting-started/sequential-model-guide/) is the easiest to get up and running with. You just declare a Sequential object and progressively add layers to it. When you're done, `compile` it with an optimizer and loss function, then fit it with some data. In this section we're going to look at a few different architectures defined with the Sequential API.

## Fully-Connected
In fully-connected (or densely-connected, or multi-layer perceptrons (MLPs)) networks, inputs to each layer are connected to every artificial neuron of the previous layer. Each neuron can be thought of as modeling a joint relationship between every MRI input voxel or every hidden layer feature. This is very powerful, but very inefficient.

![Fully-connected](https://amnesia.cbrain.mcgill.ca/deeplearning/img/fc.png)

The following example is just a [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) classifier, that takes every pixel as input and has a single layer with a [softmax](https://en.wikipedia.org/wiki/Softmax_function) activation, which is just a normalized [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) function so that it sums across the classes to 1 and we can call it a probability.

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

# clear the GPU memory in case it's loaded up from a previous experiment
from keras import backend as K
K.clear_session()

dense_model = Sequential()

dense_model.add(Flatten(input_shape=(224, 224, 3)))     # first layers of Sequential models need this input_shape argument
dense_model.add(Dense(n_classes, activation='softmax'))

dense_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
dense_model.summary()      # prints out a summary of the computational graph

dense_model.fit(image_data_train, labels_train, epochs=10, shuffle=True)

This doesnt work very well, and doesn't appear to be training at all.

![we need to go deeper](https://i.kym-cdn.com/photos/images/newsfeed/000/531/557/a88.jpg)

Let's add another couple of layers.

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

from keras import backend as K
K.clear_session()

dense_model = Sequential()

dense_model.add(Flatten(input_shape=(224, 224, 3)))
dense_model.add(Dense(64))
dense_model.add(Dense(64))
dense_model.add(Dense(n_classes, activation='softmax'))

dense_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

dense_model.summary()

# can use validation_split argument to automatically create a validation set
dense_model.fit(image_data_train, labels_train, batch_size=64, epochs=10, validation_split=0.1, shuffle=True)

OK maybe going deeper isn't always the solution.

![The Number of Parameters is Too Damn High](https://amnesia.cbrain.mcgill.ca/deeplearning/img/too_damn_high.png)



## Convolutional Neural Network

Let's define our first convolutional neural network. We're going to stack a few rectifying (reLU) [Conv2D](https://keras.io/layers/convolutional/#conv2d) layers and [MaxPool2D](https://keras.io/layers/pooling/#maxpooling2d) layers to learn some feature extractor, then train a 2-layer [Dense](https://keras.io/layers/core/#dense) classifier for prediction. The single [Batch Normalization](https://keras.io/layers/normalization/) layer really helps here, too.

![Non Linearities](https://amnesia.cbrain.mcgill.ca/deeplearning/img/non-linearities.png)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization
from keras.layers import Activation

from keras import backend as K
K.clear_session()

my_model = Sequential()

my_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
my_model.add(BatchNormalization())
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(64, kernel_size=3))
my_model.add(Activation('relu'))           # you can also add activations independently
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(64, kernel_size=3, activation='relu'))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(64, kernel_size=3, activation='relu'))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(64, kernel_size=3, activation='relu'))
my_model.add(MaxPool2D(pool_size=2))

# at this point in the model, the spatial arrangement of filter outputs
# is still present and we need to predict a vector of size (2), so we Flatten

my_model.add(Flatten())

# now we have extracted features from the data, and we need to classify it.
# we can think of this last part as a multi-layer perceptron
# most of the parameters in the model are here

my_model.add(Dense(64, activation='relu'))
my_model.add(Dense(64, activation='relu'))
my_model.add(Dense(n_classes, activation='softmax'))

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

my_model.summary()

my_model.fit(image_data_train, labels_train, batch_size=64, epochs=5, validation_split=0.1, shuffle=True)

Keras Models have a bunch of different methods other than the magical `.fit`. If you want to test whether your model works on a held-out test set, you can use [.evaluate ](https://)to return the loss, and any [metrics](https://keras.io/metrics/) that you compiled the model with. `.predict` returns probabilities of the classes you trained your model to predict, which you can turn into categorical labels (0 for undefaced, 1 for defaced) with `np.argmax()`.

In [ ]:
score = my_model.evaluate(image_data_test, labels_test)
print(my_model.metrics)
print('Loss:', score[0])
print('Test accuracy:', score[1])

prediction_probabilities = my_model.predict(image_data_test)
predictions = np.argmax(prediction_probabilities, axis=-1)

from sklearn.metrics import confusion_matrix

y_true = np.argmax(labels_test, axis=-1)

def plot_confusion(y_true, y_pred):
  conf_matrix = confusion_matrix(y_true, y_pred)
  print('Confusion matrix shape:', conf_matrix.shape)
  
  plt.imshow(conf_matrix)
  plt.xticks([0, 1], ['Undefaced', 'Defaced'], fontsize=16)
  plt.yticks([0, 1], ['Undefaced', 'Defaced'], fontsize=16)
  plt.ylabel('Truth', fontsize=20)
  plt.xlabel('Prediction', fontsize=20)
  plt.colorbar()

  plt.grid(False)
  plt.show()

plot_confusion(y_true, predictions)

Hurray, it appears to work on our test set too!

![Non Linearities](https://amnesia.cbrain.mcgill.ca/deeplearning/img/gandalf.png)

What happens if we flip the images though? It is conceivable that we'd want to use this on images from other datasets, which might not have the same orientations.

In [ ]:
flipped_test_images = image_data_test[:, :, ::-1, :]

plt.imshow(flipped_test_images[4, ...])
plt.axis('off')
plt.show()

score = my_model.evaluate(flipped_test_images, labels_test)
print('Accuracy:', score[1])

test_probs = my_model.predict(flipped_test_images)
test_predictions = np.argmax(test_probs, axis=-1)

plot_confusion(y_true, test_predictions)

FAIL

![one does not simply train a network that will generalize across datasets](https://amnesia.cbrain.mcgill.ca/deeplearning/img/one_does_not.png)

To start to address this, we can use data augmentation, to cover more of the distribution of MRI than we actually have access to. For computer vision, this usually involves random flipping and affine transformations. For neuroimaging data it is not always as clear how you would do this. Keras has [built-in data generators for common data types](https://keras.io/preprocessing/image/). We can use the image one to randomly apply transformations to our dataset for training.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_img_gen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     shear_range=0.02,
#     zoom_range=0.02,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255
)

batch_size = 64

train_generator = train_img_gen.flow_from_directory(
    'slices',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['original', 'defaced']
)

my_model.fit_generator(
    train_generator,
    steps_per_epoch=n_images // batch_size,
    epochs=5
)

# this is cheating now ("double-dipping") because we've used our test set
# as part of the training set, because the generator is itself loading all
# the images from the file system
scores = my_model.evaluate(flipped_test_images, labels_test)
print('Test accuracy:', scores[1])

class_probs = my_model.predict(flipped_test_images)

y_true = np.argmax(labels_test, axis=-1)
y_pred = np.argmax(class_probs, axis=-1)

plot_confusion(y_true, y_pred)

These generators are also very useful because in general, your datasets will probably not fit into RAM, let alone GPU memory, so you can't pass your entire training dataset into `.fit`. Here's an example of how you could write your own generator to load data on-the-fly, and Keras will do the multi-threading for you.

![alt text](https://amnesia.cbrain.mcgill.ca/deeplearning/img/gpus.png)

In [ ]:
def my_generator(image_data, labels, batch_size=32):
      n_images = image_data.shape[0]
    
      x = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
      y = np.zeros((batch_size, 2), dtype=np.int8)
      
      indices = np.asarray(range(n_images))
      
      # this loop is for one epoch
      while True:
          np.random.shuffle(indices)

          batch_idx = 0
          for i, image_index in enumerate(indices):
              batch_idx = i % batch_size
              image = image_data[image_index, ...]
              
              # randomly flip image              
              if np.random.rand() > 0.5:
                image = np.flipud(image)
              if np.random.rand() > 0.5:
                image = np.fliplr(image)
            
              x[batch_idx, ...] = image
              y[batch_idx] = labels[image_index, :]
              
              if i % batch_size == 0 and i != 0:
                yield (x, y)

If working with MRI images, you could use [nibabel](http://nipy.org/nibabel/) to load your data in a generator loop like this, or [mne-python](https://martinos.org/mne/stable/index.html) if using EEG/MEG data, or [OpenSlide](https://openslide.org/) for histology data.

I usually load the entirety of my dataset and dump it into a gigantic HDF5 file. HDF5 is a nice format, because it lets you load only parts of files into memory. This makes it far more efficient to sample if you don't need entire data examples, which are typically huge in neuroimaging. [h5py](https://www.h5py.org/) is one library that makes manipulating HDF5 files easy.

Now we can train with our custom generator using the `.fit_generator` method.

In [ ]:
my_model.fit_generator(
    my_generator(image_data_train, labels_train),
    steps_per_epoch=n_images // batch_size,
    epochs=5
)

![alt text](https://amnesia.cbrain.mcgill.ca/deeplearning/img/over-9000.png)

Here's a recipe that usually works pretty well for image/volumetric data:

1.   Stack: (1) ReLU 3x3(x3) Convolutional, (2) Batch Normalization, (3) Dropout and (4) Max Pooling layers.
2.   Repeat four or five times
3.   Flatten the output
4.   Add two fully-connected ReLU layers followed by Dropout layers
5.   Final softmax classification layer with the number of units equal to the number of classes

Train with [Adam](https://keras.io/optimizers/) optimizer, with the learning rate set an order of magnitude less than the default. 

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPool2D, BatchNormalization
from keras.layers import Activation
from keras.optimizers import Adam

from keras import backend as K
K.clear_session()

my_model = Sequential()

my_model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(224, 224, 3)))
my_model.add(BatchNormalization())
# my_model.add(Dropout(0.5))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(32, kernel_size=3, activation='relu'))
# my_model.add(BatchNormalization())
# my_model.add(Dropout(0.5))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(32, kernel_size=3, activation='relu'))
# my_model.add(BatchNormalization())
# my_model.add(Dropout(0.5))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(32, kernel_size=3, activation='relu'))
# my_model.add(BatchNormalization())
# my_model.add(Dropout(0.5))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Conv2D(32, kernel_size=3, activation='relu'))
# my_model.add(BatchNormalization())
# my_model.add(Dropout(0.5))
my_model.add(MaxPool2D(pool_size=2))

my_model.add(Flatten())

# now we have extracted features from the data, and we need to classify it
# we can think of this last part as a multi-layer perceptron
# most of the trainable parameters in the model are here

my_model.add(Dense(64, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(64, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Dense(n_classes, activation='softmax'))

# the Adam optimizer usually works well for convolutional neural networks.
# the learning rate usually needs to be tuned; usually an order of magnitude
# less than the default works well.
#
# the 'decay' parameter is an L2 regularizer on the value of the parameters,
# which constrains them from growing and growing and growing out of control
# until you end up with numerical issues
optimizer  = Adam(lr=0.0002, decay=1e-5)

my_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

my_model.summary()

[Callbacks](https://keras.io/callbacks/) allow you to call functions every epoch. There are some very useful built-in ones, like the `ModelCheckpoint`, which will save the best model found during training.


In [ ]:
from keras.callbacks import ModelCheckpoint

best_model_callback = ModelCheckpoint(filepath='/best_model.pkl', monitor='val_loss', save_best_only=True)
my_model.fit(image_data_train, labels_train, batch_size=32, epochs=10, callbacks=[best_model_callback], validation_split=0.1)

# load the best model found during training
from keras.models import load_model
my_model = load_model('/best_model.pkl')

# 4. Functional API

The [functional API ](https://keras.io/getting-started/functional-api-guide/) is a little bit more complicated, but far more flexible. It allows you to do things like have multiple inputs/outputs to a network and split/merge branches. You might want to do this if you wanted to use both imaging data and clinical metadata as inputs, or jointly predict several different clinical measures from imaging data.

We're not going to do this here, but it's also necessary for using pre-trained networks, which we'll get to.

Basically layers (or groups of layers) are now functions, and then to specify that you want to pass the output of a layer `L1` into layer `L2`, you pass `L1` as you would a variable into a normal Python function:

`L1_output = L2(L1)`

You then chain these all together to produce the entire deep network, and tell a `Model` object what the input(s) and output is.

In [ ]:
from keras.models import Model
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPool2D, BatchNormalization
from keras.layers import Input

from keras.optimizers import Adam

from keras import backend as K
K.clear_session()

# an Input layer is required by the functional API
inputs = Input(shape=(224, 224, 3))

# layers are treated as functions, and are "called" with inputs, which are the
# outputs of previous layers
conv1 = Conv2D(32, kernel_size=3, activation='relu')(inputs)
norm1 = BatchNormalization()(conv1)
drop1 = Dropout(0.5)(norm1)
pool1 = MaxPool2D(pool_size=2)(drop1)

conv2 = Conv2D(32, kernel_size=3, activation='relu')(pool1)
norm2 = BatchNormalization()(conv2)
drop2 = Dropout(0.5)(norm2)
pool2 = MaxPool2D(pool_size=2)(drop2)

conv3 = Conv2D(32, kernel_size=3, activation='relu')(pool2)
norm3 = BatchNormalization()(conv3)
drop3 = Dropout(0.5)(norm3)
pool3 = MaxPool2D(pool_size=2)(drop3)

conv4 = Conv2D(32, kernel_size=3, activation='relu')(pool3)
norm4 = BatchNormalization()(conv4)
drop4 = Dropout(0.5)(norm4)
pool4 = MaxPool2D(pool_size=2)(drop4)

conv5 = Conv2D(32, kernel_size=3, activation='relu')(pool4)
norm5 = BatchNormalization()(conv5)
drop5 = Dropout(0.5)(norm5)
pool5 = MaxPool2D(pool_size=2)(drop5)

flat = Flatten()(pool5)

fc1 = Dense(64, activation='relu')(flat)
drop_fc1 = Dropout(0.5)(fc1)
fc2 = Dense(64, activation='relu')(drop_fc1)
drop_fc2 = Dropout(0.5)(fc2)

out = Dense(n_classes, activation='softmax')(drop_fc2)


optimizer = Adam(lr=0.002, decay=1e-5)

my_functional_model = Model(inputs=[inputs], outputs=[out])

my_functional_model.summary()

my_functional_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

my_functional_model.fit(image_data_train, labels_train, batch_size=64, epochs=10, validation_split=0.1, shuffle=True)

## Transfer Learning

Many famous models are available in Keras to be re-used on other problems as [Applications](https://keras.io/applications/). They are mostly available trained on [ImageNet](http://www.image-net.org/), a computer vision competition database that contains over 14 million images that are hierarchically categorized according to the [WordNet](https://wordnet.princeton.edu/) linguistic ontology. The most common competition is ImageNet-1000, where the images are categorized in 1000 different categories.

But we don't have 1000 different categories, we have 2, and they don't correspond to the cats or dogs that make up the ImageNet categories, so we have to make some modifications. Here's the original VGG16 network that we're going to use:

![VGG16 Architecture](https://www.cs.toronto.edu/~frossard/post/vgg16/vgg16.png)

Source: [Simonyan, Karen, and Andrew Zisserman. "Very deep convolutional networks for large-scale image recognition." arXiv preprint arXiv:1409.1556 (2014)](https://arxiv.org/abs/1409.1556)

The reason that this tutorial uses 224x224 images with 3 colour channels is so that this example works. Also regular images are far easier to display with 3 channels; there are no 1-channel memes. Normally, single modalities of structural MRI scans have only 1 channel: a single voxel intensity value. This can be very confusing, as you might have to artificially expand your own data's dimensionality to include this channel dimension. Most structural MRI images are 3D `(height, width, depth)`, but you would have to train a network with input batches of size `(batch_size, height, width, depth, channels)`

In [ ]:
from keras.models import Model

from keras.layers import Dense
from keras.optimizers import Adam

from keras import backend as K
K.clear_session()

from keras.applications.vgg16 import VGG16

# here we specify that we don't want to include the top part of the network
# that contains the final layers that predict the ImageNet1000 categories.
# Keras also has an optional pooling argument for what to do if the top is not
# included. Here, we use Global Average Pooling, which averages the activations
# of each convolutional filter in the previous layer.
# This is like Flatten(), but you lose information.
pretrained_model = VGG16(include_top=False, weights='imagenet', pooling='avg', input_shape=(224, 224, 3))

# turn off training for the feature extraction portion
for layer in pretrained_model.layers:
  layer.trainable = False

# add a couple of layers using the functional API 
dense1 = Dense(128, activation='relu')(pretrained_model.output)
dense2 = Dense(128, activation='relu')(dense1)

defacing_prediction = Dense(2, activation='softmax')(dense2)

# the 'decay' parameter is an L2 regularizer on the value of the parameters,
# which constrains them from growing and growing and growing out of control
# until you end up with numerical issues
optimizer  = Adam(lr=0.0002, decay=1e-5)

transfer_model = Model(inputs=pretrained_model.input, output=defacing_prediction)

transfer_model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

transfer_model.summary()

There's a hidden callback called `History` that is called automatically, and tracks the loss function and whatever `metrics` you tell the model to compute. When you call any train method, it returns the history, which can be used to visualize the training dynamics. This is super useful to decide if you need to tweak your optimizer's learning rate, if your model is overfitting (when the validation accuracy starts to get worse but training accuracy improves)

In [ ]:
hist = transfer_model.fit(
    x=image_data_train,
    y=labels_train,
    batch_size=64,
    epochs=5,
    validation_split=0.1,
    shuffle=True
)

plt.plot(hist.history['acc'], label='train')
plt.plot(hist.history['val_acc'], label='val')
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 5. Autoencoders

In this little example, we're not going to try to predict `D`, we're going to train a little network to compress our data and learn an encoding that will reconstruct the input. This is a little bit like Principle Component Analysis, except much more powerful. A single-layer autoencoder with linear activation can learn the same dimensionality reduction as PCA, but you can also use more units, less units, add noise to the inputs, and of course go much deeper.

![alt text](https://amnesia.cbrain.mcgill.ca/deeplearning/img/brace-yourselves.png)

Here's an example of a convolutional autoencoder.

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(224, 224, 3))

x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(input_img)
x = MaxPooling2D(pool_size=2, padding='same')(x)
x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, padding='same')(x)
x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(x)
encoded = MaxPooling2D(pool_size=2, padding='same')(x)

x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(encoded)
x = UpSampling2D(size=2)(x)
x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(x)
x = UpSampling2D(size=2)(x)
x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(x)
x = UpSampling2D(size=2)(x)
decoded = Conv2D(3, kernel_size=3, activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.summary()

autoencoder.fit(image_data, image_data,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_split=0.1
               )

In [ ]:
prediction = autoencoder.predict(image_data[0:1, ...])

f, axes = plt.subplots(1, 2)
axes[0].imshow(image_data[0, ...])
axes[1].imshow(prediction[0, ...])
axes[0].grid('off')
axes[1].grid('off')
plt.show()

It doesn't really work too well and I didn't spend very much time on this example. Increasing the accuracy is left as an exercise to the reader.

# 6. Interpretability
What are these deep networks even learning? This section is going to explore a few different ways to begin to understand the models, and why they make predictions.

![Everyone gets a heatmap](https://amnesia.cbrain.mcgill.ca/deeplearning/img/and-then.png)

 In the first section, we're going to use the `keras-vis` package, which creates visualizations for the purposes of interpreting predictions from neural networks. There's another package, `innvestigate`, that implements LRP, but it's not as well documented. We're going to go through a few techniques here:

1.   Filter Activations
2.   Class Appearance Models
3.   Activation Maximization
4.   LIME
5.   Layer-wise Relevance Propagation

This section assumes that `my_model` is a trained model. OK let's get started interpreting what it learned. Let's start by visualizing the activations of filters deeper into our network.

## Filter Activations

Here's how you can show, for some input, the activation of some filter anywhere in the network. There are a lot of these filters.

In [ ]:
import keras.backend as K

def show_activation(input_image, layer_name, filter_idx):
    
    # create a structure to more easily access layers by name
    layer_lookup = dict([(layer.name, layer) for layer in my_model.layers])

    layer_output = layer_lookup[layer_name].output

    # create a function that stops the forward pass at the layer we want
    partial_net = K.function([my_model.input], [layer_output]

    # add a batch dimension to the image
    input_image = input_image[np.newaxis, ...]
    
    activation = partial_net([input_image])[0] 
    activation = activation[0, :, :, filter_idx]
    
    f, axes = plt.subplots(1, 2)
    axes[0].imshow(input_image[0, ...])
    axes[1].imshow(activation)
    axes[0].grid('off')
    axes[1].grid('off')

    plt.show()

In [ ]:
# print the names of the layers in our model
for layer in my_model.layers:
  print(layer.name)
  
for layer in my_model.layers:
  if 'conv' in layer.name:
    show_activation(image_data_train[0, ...], layer.name, 0)
#     show_activation(image_data_train[0, ...], layer.name, 1)

This gets uninterpretable pretty quickly, and the deeper you go, the less clear it is what is happening. Let's try some more complicated methods. First, install `keras-vis`, a great package that implements saliency, Grad-CAM, and Class Appearance Models:

In [ ]:
!pip install git+https://github.com/raghakot/keras-vis.git

[TensorFlow](https://www.tensorflow.org/) is the default backend of Keras, and TensorFlow's computational graph is static after you've compiled it. This is a problem for many interpretability methods because they involve propagating the gradient back to the input, which is not possible with a softmax layer at the end, because the softmax gets rid of the gradient by squashing the outputs so that they sum to 1. This means that each output depends on all of the weights, which is bad for gradients!

The first step is to use a `keras-vis` utility to remove the last layer (softmax), and replace it with a linear one.

In [ ]:
from vis.utils import utils
from keras import activations

if my_model.layers[-1].activation != activations.linear:
  print('Modifying computational graph...')
  my_model.layers[-1].activation = activations.linear
  my_model = utils.apply_modifications(my_model)
else:
  print('Last layer is alreary linear')
print('Done!')

## Class Appearance Model

We're going to generate the input that the model thinks is the most representative single input for each class.

In [ ]:
from vis.utils import utils
from keras import activations
import matplotlib.cm as cm

from vis.visualization import visualize_activation

layer_idx = -1

print('Computing faced activation...')
faced_activation = visualize_activation(my_model, layer_idx, filter_indices=0)
print('Computing defaced activation...')
defaced_activation = visualize_activation(my_model, layer_idx, filter_indices=1)

# plot results
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(faced_activation)
axes[1].imshow(defaced_activation)
axes[0].grid('off')
axes[1].grid('off')
plt.show()

This is.... not obvious. There are other ways to do this, but it's never obvious how much information about the model you can extract from them.

You can use algorithms like DeepDream to do "gradient ascent" at the output or at intermediate layers to do the same kind of thing. More information:
*   [Deep Dream notebook](https://colab.research.google.com/drive/1DWcrN9WXni58MbddvlShX0wF_oeo8W_0)
*   [Visualizing Features with Keras](https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html)
*   [Feature Visualization Distill.pub article](https://distill.pub/2017/feature-visualization/)

# Individual Predictions

Here are some ways you can explain individual predictions.

![Everyone gets a heatmap](https://amnesia.cbrain.mcgill.ca/deeplearning/img/you-get-a-heatmap.png)

## Saliency
Saliency is all about the Jacobian, and a way to do sensitivity analysis. It uses the derivative of the output prediction with respect to each input pixel.

You can also do this for hidden layers! The second parameter in `visualize_saliency` chooses the index of the layer.

Paper: [https://arxiv.org/abs/1312.6034](https://arxiv.org/abs/1312.6034)

In [ ]:
from vis.visualization import visualize_saliency, overlay
import matplotlib.cm as cm

sample_idx = 1
class_idx = int(labels[sample_idx])

test_image = image_data[sample_idx, ...]
display_image = np.uint8(test_image * 255)

print('Computing visual saliency...')
saliency_gradients = visualize_saliency(my_model, -1, filter_indices=class_idx, seed_input=test_image)
saliency_heatmap = np.uint8(cm.jet(saliency_gradients)[..., :3] * 255)
saliency_overlay = overlay(saliency_heatmap, display_image)

# plot output
fig, axes = plt.subplots(1, 3, figsize=(12, 8))
axes[0].imshow(test_image)
axes[1].imshow(saliency_gradients, cmap='jet')
axes[2].imshow(saliency_overlay)

for ax in axes:
  ax.set_xticks([])
  ax.set_yticks([])
plt.show()


Lots of saliency in the face area! Looks good! How about for the other class?

In [ ]:
from vis.visualization import visualize_saliency, overlay
import matplotlib.cm as cm

sample_idx = 1
class_idx = labels[sample_idx]
other_class_idx = np.abs(1 - class_idx)  # saliency for other class

test_image = image_data[sample_idx, ...]
display_image = np.uint8(test_image * 255)

print('Computing visual saliency...')
saliency_gradients = visualize_saliency(my_model, -1, filter_indices=other_class_idx, seed_input=test_image)
saliency_heatmap = np.uint8(cm.jet(saliency_gradients)[..., :3] * 255)
saliency_overlay = overlay(saliency_heatmap, display_image)

# plot output
fig, axes = plt.subplots(1, 3, figsize=(12, 8))
axes[0].imshow(test_image)
axes[1].imshow(saliency_gradients, cmap='jet')
axes[2].imshow(saliency_overlay)

for ax in axes:
  ax.set_xticks([])
  ax.set_yticks([])
plt.show()

![This is fine](https://amnesia.cbrain.mcgill.ca/deeplearning/img/this-is-fine.png)

## Gradient-weighted Class Activation Map

This example computes the Grad-CAM gradient-weighted class activation map.

In [ ]:
from vis.visualization import visualize_cam, overlay

sample_idx = 1
class_idx = int(labels[sample_idx])

test_image = image_data[sample_idx, ...]
display_image = np.uint8(test_image * 255)

print('Computing class activation...')
cam_gradients = visualize_cam(my_model, -1, filter_indices=class_idx, seed_input=test_image, backprop_modifier=None)
cam_heatmap = np.uint8(cm.jet(cam_gradients)[..., :3] * 255)
cam_overlay = overlay(cam_heatmap, display_image)

print('Grad-CAM for class:', class_idx)
# plot output
fig, axes = plt.subplots(1, 3, figsize=(12, 8))
axes[0].imshow(test_image)
axes[1].imshow(cam_gradients, cmap='jet')
axes[2].imshow(cam_overlay)
for ax in axes:
  ax.set_xticks([])
  ax.set_yticks([])
plt.show()

Let's try multiplying the Grad-CAM with Guided Backprop. This is just the `backprop_modifier` parameter to `visualize_cam`.

In [ ]:
from vis.visualization import visualize_cam, overlay

sample_idx = 1
class_idx = int(labels[sample_idx])

test_image = image_data[sample_idx, ...]
display_image = np.uint8(test_image * 255)

print('Computing class activation...')
cam_guided_gradients = visualize_cam(my_model, -1, filter_indices=class_idx, seed_input=test_image, backprop_modifier='guided')
cam_heatmap = np.uint8(cm.jet(cam_guided_gradients)[..., :3] * 255)
cam_overlay = overlay(cam_heatmap, display_image)

# plot output
fig, axes = plt.subplots(1, 3, figsize=(12, 8))
axes[0].imshow(test_image)
axes[1].imshow(cam_guided_gradients, cmap='jet')
axes[2].imshow(cam_overlay)
for ax in axes:
  ax.set_xticks([])
  ax.set_yticks([])
plt.show()

![alt text](https://amnesia.cbrain.mcgill.ca/deeplearning/img/not-sure.png)

## LIME
This method does some feature selection and then starts with the assumption that even highly non-linear classifiers, around the space of the input, can be approximated as linear. This means that small perturbations of the input shouldn't result in huge changes in the output.

It relies on feature selection (and selecting a method to select features, which in and of itself is a challenge), and is not always 100% stable, but here it is!

GitHub repository with high-level explanation of how it works: [https://github.com/marcotcr/lime](https://github.com/marcotcr/lime)

Paper: [https://arxiv.org/abs/1602.04938](https://arxiv.org/abs/1602.04938)

In [ ]:
!pip install lime

First, let's take our trained classifier and try to predict whether there's a face in some images:

In [ ]:
sample_idx = 1

# when using .predict, input data shape has to be: (batch_size, dim1, ..., dimN, n_channels)
# this can be super confusing, keeping track of where the batch dimension and where the channel dimension is

# if we take a slice of size 1 as opposed to 0, we can preserve that numpy dimension
test_image = image_data[sample_idx:sample_idx+1, ...]
print('Batch shape:', test_image.shape)

predictions = my_model.predict(test_image)
print('Model predictions:', predictions)

In [ ]:
import lime
from lime import lime_image

sample_idx = 1

test_image = image_data[sample_idx, ...]
class_idx = labels[sample_idx]

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(test_image, my_model.predict, top_labels=2, hide_color=0, num_samples=1000)

In [ ]:
def show_lime_explanation(original_image, explanation_image, explanation_mask):
  from skimage.segmentation import mark_boundaries
  
  fig, axes = plt.subplots(1, 2, figsize=(12, 6))
  axes[0].imshow(original_image)
  axes[1].imshow(mark_boundaries(explanation_image, explanation_mask))
  for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
  plt.show()

Let's try some stuff

In [ ]:
print('Showing LIME evidence for faced class')
explanation_image, explanation_mask = explanation.get_image_and_mask(0, positive_only=True, num_features=5, hide_rest=True)
show_lime_explanation(test_image, explanation_image, explanation_mask)

print('Showing LIME evidence for defaced class')
explanation_image, explanation_mask = explanation.get_image_and_mask(1, positive_only=True, num_features=5, hide_rest=True)
show_lime_explanation(test_image, explanation_image, explanation_mask)

print('Showing LIME evidence for faced class, including negative evidence')
explanation_image, explanation_mask = explanation.get_image_and_mask(0, positive_only=False, num_features=5, hide_rest=False)
show_lime_explanation(test_image, explanation_image, explanation_mask)

LIME's `ImageExplainer` class uses "super pixels", which groups image regions into a small number of regions. This implementation uses some combination of distance (regions have to be connected) and pixel intensity. How many regions are there? How similar do pixels have to be? You could probably play with these parameters until you get a nice looking explanation.

![This is fine](https://amnesia.cbrain.mcgill.ca/deeplearning/img/cant-be-wrong.png)

## Layerwise Relevance Propagation

LRP is probably the most rigourously mathematically supported linear approximation for importance, but also the most difficult to understand fully. There are many choices of relevant propagation rule, and choices must be made for different types of non-linearity for how to propagate each non-linear region of the function. The authors even suggest different propagation rules for different non-linearities.

iNNvestigate GitHub repository that implements it and other interpretation methods for Keras models: [https://github.com/albermax/innvestigate](https://github.com/albermax/innvestigate)

LRP Paper: [https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0130140](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0130140)

CVPR Tutorial Video: [https://www.youtube.com/watch?v=LtbM2phNI7I](https://www.youtube.com/watch?v=LtbM2phNI7I)

In [ ]:
!pip install git+https://github.com/albermax/innvestigate

![This is fine](https://amnesia.cbrain.mcgill.ca/deeplearning/img/grinds-gears.png)


In [ ]:
import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis

from keras import activations

if my_model.layers[-1].activation != activations.linear:
  model_wo_softmax = iutils.keras.graph.model_wo_softmax(my_model)
else:
  model_wo_softmax = my_model

lrp_analyzer = innvestigate.create_analyzer("lrp.z", model_wo_softmax)

image_batch = image_data[0:1, ...]

analysis = lrp_analyzer.analyze(image_batch)
heatmap = ivis.heatmap(analysis)


f, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(image_data[0])
axes[1].imshow(heatmap[0])
axes[0].axis('off')
axes[1].axis('off')
plt.show()

The end.

![done](https://amnesia.cbrain.mcgill.ca/deeplearning/img/done.png)